Google Database

Use [Google Memorystore for Redis](https://cloud.google.com/memorystore/docs/redis/memorystore-for-redis-overview) to store Documents for LangChain.

## Pre-reqs

### Setting Up a Memorystore for Redis Instance

Before proceeding, an active Memorystore for Redis instance is needed to store chat message history:

* Create a Memorystore for Redis Instance (>= 5.0): If an instance doesn't exist, follow the instructions at https://cloud.google.com/memorystore/docs/redis/create-instance-console to create a new one. Ensure the version is greater than or equal to 5.0.
* Obtain Endpoint: Note the endpoint associated with the instance.

### Installing the LangChain Memorystore for Redis Module

Interaction with the Memorystore for Redis instance from LangChain requires installing the necessary module:

In [ ]:
# Install Memorystore for Redis for LangChain module
%pip install langchain_google_memorystore_redis

## Basic Usage

### Initialize a MemorystoreDocumentLoader

Initialize a loader that loads all documents stored in the Memorystore for Redis instance with a specific prefix.

In [ ]:
import redis
from langchain_google_memorystore_redis import MemorystoreDocumentLoader

# Connect to a Memorystore for Redis instance
redis_client = redis.from_url("redis://127.0.0.1:6379")
prefix = "doc:"

loader = MemorystoreDocumentLoader(
    client=redis_client,
    key_prefix=prefix,
    content_fields=set(["page_content"]),
)

### Load Documents

Load all documents stored in the Memorystore for Redis instance at once.

In [ ]:
documents = loader.load()

### Lazy Load Documents

Load the document one-by-one with lazy_load generator.

In [ ]:
for document in loader.lazy_load():
  # Do something
  print(document)

## Customize Document Page Content & Metadata

When initializing a loader with more than 1 content field, the `page_content` of the loaded Documents will contain a JSON-encoded string with top level fields equal to the specified fields in `content_fields`.

If the `metadata_fields` are specified, the `metadata` field of the loaded Documents will only have the top level fields equal to the specified `metadata_fields`. If any of the values of the metadata fields is stored as a JSON-encoded string, it will be decoded prior to being loaded to the metadata fields.

In [ ]:
loader = MemorystoreDocumentLoader(
    client=redis_client,
    key_prefix=prefix,
    content_fields=set(["content_field_1", "content_field_2"]),
    metadata_fields=set(["title", "author"]),
)

## Save Documents

You can save a list of Documents into Memorystore for Redis instance like below. The Documents will be stored into randomly generated keys with the specified prefix of `key_prefix`. Alternatively, you can designate the suffixes of the keys by specifying `ids` in the `add_documents` method.

In [ ]:
from langchain_google_memorystore_redis import MemorystoreDocumentSaver

# Connect to a Memorystore for Redis instance
redis_client = redis.from_url("redis://127.0.0.1:6379")
prefix = "doc:"

saver = MemorystoreDocumentSaver(
    client=redis_client,
    key_prefix=prefix,
    content_field="page_content",
)
saver.add_documents(documents)

### Clean up Saved Documents

Delete all of keys with the specified prefix in the Memorystore for Redis instance. You can also specify the suffixes of the keys if you know.



In [ ]:
saver.delete()